In [150]:
import ast
import requests
import json
from datetime import datetime , timedelta
import time
import pandas as pd
import numpy as np

headername = ['crypto','date','time','high','low','open','close','gap','from_time','to_time']


cryptoprice_set = pd.DataFrame(columns=headername)

total_cryptoprice_set = pd.DataFrame(columns=headername)

newHeaders = {'Content-type': 'application/json', 'Accept': 'text/plain'}

crypto = 'BCH'

datestr = '2018-12-20'

print(type(datestr))



<class 'str'>


In [151]:
yyyy = int(datestr[0:4])

mm = int(datestr[5:7])

dd = int(datestr[8:10])

#datestr = str(yyyy)+'-'+str(mm)+'-'+'dd'

print(yyyy)
print(mm)
print(dd)

2018
12
20


In [152]:
dt = datetime(yyyy, mm, dd, 23, 23)
print("Unix Timestamp: ",(time.mktime(dt.timetuple())))
targetTime = (time.mktime(dt.timetuple()))


Unix Timestamp:  1545315780.0


In [153]:
url = "https://min-api.cryptocompare.com/data/v2/histohour?fsym="+crypto+"&tsym=USD&limit=24&aggregate=1&toTs="+str(targetTime)

print(url)

response_Json = requests.get(url=url,headers=newHeaders)

https://min-api.cryptocompare.com/data/v2/histohour?fsym=BCH&tsym=USD&limit=24&aggregate=1&toTs=1545315780.0


In [154]:
jsonFile = response_Json.json()

In [155]:
jsonData = jsonFile['Data']['Data']

In [156]:


datarow = []

for entry in jsonData : 
    ptime = entry.pop('time')
    
    ptime = datetime.fromtimestamp(
        int(ptime)
    ).strftime('%Y-%m-%d %H:%M:%S')
    
    phigh = entry.pop('high')
    plow = entry.pop('low')
    popen = entry.pop('open')
    pclose = entry.pop('close')

    # pvolumefrom = entry.pop('volumefrom')
    # pvolumeto = entry.pop('volumeto')

    #pconversionType = entry.pop('conversionType')
    #pconversionSymbol = entry.pop('conversionSymbol')

    datarow.append(crypto)
    datarow.append(datestr)

    datarow.append(ptime)
    datarow.append(phigh)
    datarow.append(plow)
    datarow.append(popen)
    datarow.append(pclose)
    datarow.append( ((pclose-popen)/popen) * 100 )
    datarow.append()
    datarow.append()

    # datarow.append(pconversionType)
    # datarow.append(pconversionSymbol)

    cryptoprice_set = cryptoprice_set.append(pd.Series(datarow,index=headername),ignore_index=True)
    datarow = []
    
#for key,val in jsonData.items() :
#    print(key ,' : ',val)

TypeError: append() takes exactly one argument (0 given)

In [ ]:
#cryptoprice_set

In [ ]:
maxgap_set = cryptoprice_set[cryptoprice_set['gap']==cryptoprice_set['gap'].max()]

In [ ]:
maxgap_set['time'].values


In [ ]:
from_time = datetime.strptime(maxgap_set.iloc[0]['time'],'%Y-%m-%d %H:%M:%S') + timedelta(hours=-6)

In [157]:
maxgap_set['from_time'] =  from_time

In [158]:
maxgap_set

,crypto,date,time,high,low,open,close,gap,from_time,to_time
18,BCH,2018-12-20,2018-12-20 17:00:00,154.01,148.37,140.25,153.08,9.14795,2018-12-20 11:00:00,2018-12-20 23:00:00


In [159]:
maxgap_set['to_time'] = datetime.strptime(maxgap_set.iloc[0]['time'],'%Y-%m-%d %H:%M:%S') + timedelta(hours=+6)

In [160]:
maxgap_set

,crypto,date,time,high,low,open,close,gap,from_time,to_time
18,BCH,2018-12-20,2018-12-20 17:00:00,154.01,148.37,140.25,153.08,9.14795,2018-12-20 11:00:00,2018-12-20 23:00:00


In [161]:
total_cryptoprice_set = pd.concat([total_cryptoprice_set,maxgap_set], axis=0, ignore_index=False)

In [162]:
total_cryptoprice_set

,crypto,date,time,high,low,open,close,gap,from_time,to_time
18,BCH,2018-12-20,2018-12-20 17:00:00,154.01,148.37,140.25,153.08,9.14795,2018-12-20 11:00:00,2018-12-20 23:00:00
